In [ ]:
import os
from pathlib import Path
import torch 
import wandb

CONFIG = dict (
    project = "hlc-polyp-detection",
    architecture = "fasterrcnn_resnext50_32x4d",
    dataset_id = "hlc-custom-polyp-detection",
    infra = "osx",
    num_classes = 2,
    max_epochs = 100,
    lr=0.01,
    min_lr=0.0000001,
    epochs=15,
    batch_size=4,
    nesterov=True,
    momentum=0.9,
    weight_decay=0.0005,
    clip_limit=0.25,
    difference=False,
    name="fancy_walrus"
)

ROOT_DIR = os.path.abspath("./")
DATA_DIR = os.path.join(ROOT_DIR, "data")
MODEL_DIR = os.path.join(ROOT_DIR, "model")
LOG_DIR = os.path.join(ROOT_DIR, "log")

NUM_CLASSES = 2

BATCH_SIZE = 4
INPUT_SIZE = 1024
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = 8

wandb_key = Path(os.path.join(ROOT_DIR, "wandb.txt")).read_text().strip()
os.environ["WANDB_API_KEY"] = wandb_key
os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(ROOT_DIR, "LightningFastRCNN-polyps.ipynb")

In [ ]:
from src.PolypsPLDataModule import PolypsPLDataModule
from src.LightningFasterModule import LightningFasterModule
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

# Initialize the data module
polyp_dm = PolypsPLDataModule(data_dir='./data', batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

# Load the model from the checkpoint
model = LightningFasterModule.load_from_checkpoint(os.path.join(MODEL_DIR, "chkpts/lazy_panda-chkpt-full.ckpt"))

# Initialize the WandB logger
wandb_logger = WandbLogger(project=CONFIG['project'], log_model="all")

# Initialize the trainer
trainer = Trainer(
    logger=wandb_logger,
    accelerator='gpu' if torch.cuda.is_available() else None,
    log_every_n_steps=1
)

# Run the test set
trainer.test(model, datamodule=polyp_dm)



